# GAxOctagon Hackathon: Data Gathering
### Danielle Medellin, Sally Huang, Stephen Burnett, Pierce Butler, Katherine Lough

## Problem Statement
With Octagon being in the events business, they are looking for a way for consumers to "check-in" contactless onsite at events. Our goal is to build a platform that allows for efficient contact-less check-ins that is both easy and quick for attendees.

Due to the recent COVID-19 pandemic, safety precautions at future events are of incredible importance. In order to inform the creation of our product, we wanted to gauge public opinion on attending events related to the sports and entertainment industries in a post-COVID world.

## Executive Summary
Due to the fact that the COVID-19 pandemic is still ongoing, it was difficult to nail down data to use for this exploration. After much consideration and searching for open data resources, we decided our data could come from the subreddit r/Coronavirus. We collected data using the [Pushshift API](https://github.com/pushshift/api). We collected comments from posts in the r/Coronavirus [subreddit](www.reddit.com/r/coronavirus). Comments from the past 60 days were looked at in an effort to get recent opinions. We queried 16 different key words related to COVID and events to filter the comments that came through.

The data collected included the following features: author, body, created_utc, subreddit, permalink, query, and timestamp. More information can be found in the data dictionary below.

### Data Dictionary
|Feature|Type|Description|
|---|:---:|:---|
|author|object|Username of the author of the comment|
|body|object|Full comment text|
|created_utc|integer|Coordinated Universal Time of comment|
|subreddit|object|Subreddit where comment was found (Coronavirus)|
|permalink|object|Partial link to the original comment|
|query|object|Query word that pulled in the comment|
|timestamp|datetime|Date of comment (YYYY-MM-DD)|

## Loading Libraries

In [1]:
import pandas as pd
import datetime as dt
import time
import requests

## Function for Data Collection Using Pushshift API

In [32]:
# adapted from Mahdi Shadkam-Farrokhi

def query_pushshift(subreddit, query_list, kind = 'comment', day_window = 60): # 30 day
    SUBFIELDS = ['author', 'body', 'created_utc', 'subreddit', 'permalink', 'query']
    
    # establish base url and stem
    BASE_URL = f"https://api.pushshift.io/reddit/search/{kind}" # also known as the "API endpoint" 
    stem1 = f"{BASE_URL}?q="
    stem2 = f"&subreddit={subreddit}&size=500&after={day_window}d" # always pulling max of 500
    
    # instantiate empty list for temp storage
    comments = []
    
    # implement for loop with `time.sleep(2)`
    for query in query_list:
        URL = "{}{}{}".format(stem1, query, stem2)
        print("Querying from: " + URL)
        response = requests.get(URL)
        assert response.status_code == 200
        mine = response.json()['data']
        df = pd.DataFrame.from_dict(mine)
        df['query'] = query
        comments.append(df)
        time.sleep(2)
    
    # pd.concat storage list
    full = pd.concat(comments, sort=False)
    
    # if submission
    if kind == "comment":
        # select desired columns
        full = full[SUBFIELDS]
        # drop duplicates
        full.drop_duplicates(inplace = True)
        

    # create `timestamp` column
    full['timestamp'] = full["created_utc"].map(dt.date.fromtimestamp)
    
    print("Query Complete!")    
    return full 

This function will use the Pushshift API to gather comments from specific subreddit pages based on keywords. It is set to look for 500 posts at a time from the past 60 days. We are pulling the following columns: `author`, `body`, `created_utc`, `subreddit`, and `permalink`. Within the function we do some cleaning by dropping any duplicate comments. Lastly, we convert the `created_utc` column into datetime form. A dataframe is created. This function will be repeated for each specific keyword or query word that we are looking for in the comments. 

In [29]:
# query list
queries = {'event','sports','safe','safety','attend','game','concert','feel','fan','protocols','measures','concern','danger','dangerous','unsafe','social', 'unsafe'}

We will collect comments from the r/Coronavirus subreddit with any of the words in the query list above. 

In [33]:
# collecting comments
events_df = query_pushshift('coronavirus',queries)

Querying from: https://api.pushshift.io/reddit/search/comment?q=measures&subreddit=coronavirus&size=500&after=60d
Querying from: https://api.pushshift.io/reddit/search/comment?q=safe&subreddit=coronavirus&size=500&after=60d
Querying from: https://api.pushshift.io/reddit/search/comment?q=game&subreddit=coronavirus&size=500&after=60d
Querying from: https://api.pushshift.io/reddit/search/comment?q=event&subreddit=coronavirus&size=500&after=60d
Querying from: https://api.pushshift.io/reddit/search/comment?q=attend&subreddit=coronavirus&size=500&after=60d
Querying from: https://api.pushshift.io/reddit/search/comment?q=danger&subreddit=coronavirus&size=500&after=60d
Querying from: https://api.pushshift.io/reddit/search/comment?q=dangerous&subreddit=coronavirus&size=500&after=60d
Querying from: https://api.pushshift.io/reddit/search/comment?q=unsafe&subreddit=coronavirus&size=500&after=60d
Querying from: https://api.pushshift.io/reddit/search/comment?q=social&subreddit=coronavirus&size=500&af

In [34]:
# check dataframe
events_df

,author,body,created_utc,subreddit,permalink,query,timestamp
0,GrauGeist8888,WTF are you even asking for? It is self-evide...,1590182777,Coronavirus,/r/Coronavirus/comments/gnuq3s/only_73_of_peop...,measures,2020-05-22
1,Disaster_Area,I'm context it's quite obvious that I was sayi...,1590183249,Coronavirus,/r/Coronavirus/comments/go0p56/potus_has_legal...,measures,2020-05-22
2,IHeedNealing,Not at all my point or intent. Just a few week...,1590183477,Coronavirus,/r/Coronavirus/comments/gojqi1/daily_discussio...,measures,2020-05-22
3,Omnitraxus,&gt;One of your aunt's colleagues is a super s...,1590183560,Coronavirus,/r/Coronavirus/comments/go9exo/the_us_records_...,measures,2020-05-22
4,srelma,&gt;Were reporting something like 140k deaths ...,1590183767,Coronavirus,/r/Coronavirus/comments/goicek/the_death_toll_...,measures,2020-05-22
...,...,...,...,...,...,...,...
95,VarkingRunesong,It sounds over the top but that’s what we do t...,1590194586,Coronavirus,/r/Coronavirus/comments/gouixw/nc_38_test_posi...,feel,2020-05-22
96,The_avocado_girl,Same with Illinois-&gt; information and plans ...,1590194670,Coronavirus,/r/Coronavirus/comments/gokm4t/a_majority_of_a...,feel,2020-05-22
97,khalifornia420,Agreed. Tbh I think a lot of the people on thi...,1590194698,Coronavirus,/r/Coronavirus/comments/gokm4t/a_majority_of_a...,feel,2020-05-22
98,ivXtreme,Even the most selfless of people have selfish ...,1590194796,Coronavirus,/r/Coronavirus/comments/gorp06/matthew_mcconau...,feel,2020-05-22


We successfully collected 1600 comments. 

In [35]:
# write to CSV 
events_df.to_csv('./data/events_reddit.csv', index=False)

Continue to the next notebook [`hackathon_analysis`](./hackathon_analysis.ipynb) for analysis of these comments. 